# Random Forest Classifier

This is a component that performs predictions using a Random Forest Classifier implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the SDK to load a model and other artifacts.
- how to use a model to provide real-time predictions.

In [ ]:
%%writefile Predictor.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Predictor(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        logger.info("Initializing")

        # Load Artifacts: Estimator, Encoders, etc
        model = load_model(experiment_id=experiment_id)
        self.estimator = model["estimator"]
        self.feature_encoder = model["feature_encoder"]
        self.label_encoder = model["label_encoder"]
        self.columns = model["columns"]
        self.datetime_indexes = model["datetime_indexes"]
        self.categorical_indexes = model["categorical_indexes"]
        self.numerical_nan_replacement = model["numerical_nan_replacement"]
        self.categorical_nan_replacement = model["categorical_nan_replacement"]

        logger.info("Init complete!")

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Returns a prediction.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.

        Returns:
            Array-like with predictions.
        """
        # Put data in a pandas.DataFrame
        df = pd.DataFrame(X, columns=feature_names)

        # Replace NaNs
        df.fillna(self.categorical_nan_replacement, inplace=True)
        df.fillna(self.numerical_nan_replacement, inplace=True)

        # Put data back in a numpy.ndarray
        X = df[self.columns].to_numpy()

        # Remove datetime features
        X = X[:, np.where(~self.datetime_indexes)[0]]

        # Encode categorical features
        if np.ma.any(self.categorical_indexes):
            X[:, self.categorical_indexes] = \
                self.feature_encoder.transform(X[:, self.categorical_indexes])

        # Perform Prediction
        y_pred = self.estimator.predict(X)

        # Apply Label Decoder
        return self.label_encoder.inverse_transform(y_pred)

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%bash
DATASET="${DATASET:-titanic}"
TARGET="${TARGET:-Survived}"
EXPERIMENT_ID="${EXPERIMENT_ID:-08d03e85-4a7c-4376-873c-5828ea603e1f}"
MODEL_NAME="Predictor"
API_TYPE="REST"
SERVICE_TYPE="MODEL"
PERSISTENCE=0
LOG_LEVEL="DEBUG"
read -r -d "" PARAMETERS << EOM
[{"type": "STRING", "name": "dataset", "value": "$DATASET"},
 {"type": "STRING", "name": "target", "value": "$TARGET"},
 {"type": "STRING", "name": "experiment_id", "value": "$EXPERIMENT_ID"}]
EOM

seldon-core-microservice "$MODEL_NAME" "$API_TYPE" \
    --service-type "$SERVICE_TYPE" \
    --persistence "$PERSISTENCE" \
    --parameters "$PARAMETERS" \
    --log-level "$LOG_LEVEL" > log.txt 2>&1 &

ATTEMPT=0
until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    # exit process if not healthy after 10 seconds
    if [ "$ATTEMPT" -gt 10 ]; then
        cat log.txt
        exit 1
    fi
    ATTEMPT=$((ATTEMPT + 1))
    sleep 1
done
echo "Deployment successful. Waiting for requests."

## Make predictions

In [ ]:
%%bash
curl -sSL localhost:5000/predict --data-binary @- << EOF
json={
    "data": {
        "names": ["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"],
        "ndarray": [
            [123, 2, "Nasser, Mr. Nicholas", "male", 32.5, 1, 0, "237736", 30.0708, null, "C"]
        ]
    }
}
EOF

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-mic | awk '{print $2}' | xargs -r kill